In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [2]:
nhl_file_path = 'nhl_data.csv'
nhl_data = pd.read_csv(nhl_file_path)

In [3]:
# dropna drops missing values (think of na as "not available")
nhl_data = nhl_data.dropna(axis=0)

In [4]:
y = nhl_data.Salary

In [5]:
nhl_features = ['Age', 'G', 'A', 'PTS', '+/-', 'Shifts']

In [6]:
X = nhl_data[nhl_features]

In [7]:
# Define model. Specify a number for random_state to ensure same results each run
nhl_model = DecisionTreeRegressor(random_state=0)

In [8]:
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
nhl_model.fit(train_X, train_y)
val_predictions = nhl_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

3047222.222222222
